In [5]:
N_COINS = 3
PRECISION = 1e18

D = 29047349899039352915342907
# USDC, WBTC, ETH
balances = [9835488234445, 14938821663, 2811863139959144531319]
precisions = [1e12, 1e10, 1]
price_scale = [66266853956842562806222, 3465399375853261468310]

xp = [b for b in balances]
xp[0] *= precisions[0]
for k in range(1, N_COINS):
    xp[k] = xp[k] * price_scale[k - 1] * precisions[k] / PRECISION

for x in xp:
    print(x / 1e18)

D / 1e18

9835488.234445
9899487.13429337
9744228.770199211


29047349.899039354

In [6]:
N_COINS = 3
PRECISION = 1e18

D = 29047349899039352915342907
balances = [9835488234445, 14938821663, 2811863139959144531319]
precisions = [1e12, 1e10, 1]
price_scale = [66266853956842562806222, 3465399375853261468310]

def get_xcp(D, prices):
    N = 3
    x = [0, 0, 0]
    x[0] = D / N

    for i in range(1, N):
        x[i] = D * 10**18 / (N * prices[i - 1])

    return (x[0] * x[1] * x[2]) ** (1/N)

get_xcp(D, price_scale) / 1e18

15811.341284314029

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import fsolve

D = 200
A = 5
g = 0.05
p = [1, 2]

def f_curve_v2(y, x, A, D, g, p):
    # Transformed balances
    x = x * p[0]
    y = y * p[1]
    # Curve v2
    k0 = x*y/(D/2)**2
    k = A*k0*g**2/(g+1-k0)**2
    return x*y + k*D*(x+y) - (D/2)**2 - k*D**2

xs = np.linspace(10, 400, 100)
ys = [fsolve(f_curve_v2, (D/2)**2, args=(x, A, D, g, p))[0] for x in xs]

### Curve v2 graph ###
plt.figure(figsize=(8, 8)) 
# plt.axis('equal')
plt.ylim(0, 300)
plt.xlim(0, 300)

plt.plot(xs, ys)

plt.grid(True)
plt.show()

### xy / (D/(2 * p0)*D/(2*p1)) ###
xys = zip(xs, ys)
zs = [x*y/((D/(2*p[0]))*(D/(2*p[1]))) for (x, y) in xys]

plt.plot(xs, zs)
plt.plot(ys, zs)
plt.legend(["x", "y"])
plt.grid(True)
plt.show()